# 🟢 Hackathon Agenti Cosmici 2025
## Round 1: Addestramento Cosmico

Benvenuto nel notebook ufficiale del Round 1!

In questo round affronterai 4 missioni per imparare a navigare, commerciare e usare le API galattiche.

**Istruzioni:**
- Completa ogni missione seguendo la traccia.
- Usa le classi `GalacticMarketplace`, `GalaxyNavigator`, `InfoSphere` da `galactic_apis.py`.
- Valuta la tua soluzione con il sistema automatico.
- Puoi modificare e rieseguire le celle tutte le volte che vuoi!

---

In [ ]:
# 📦 SETUP: Import e inizializzazione
import json
from galactic_apis import GalacticMarketplace, GalaxyNavigator, InfoSphere
from evaluation_system import HackathonEvaluator, display_evaluation_results

# Carica stato galattico iniziale
with open('ROUND 1 FILES/galaxy_state.json') as f:
    galaxy_state = json.load(f)

# Inizializza API
marketplace = GalacticMarketplace(galaxy_state)
navigator = GalaxyNavigator(galaxy_state)
infosphere = InfoSphere(galaxy_state)

# Inizializza valutatore
evaluator = HackathonEvaluator(round_number=1)


---

## Missione 1
**Trova dove si trova R2-D2 e portalo su Coruscant usando la nave più economica**

Suggerimento: usa `navigator.get_droid_location`, `navigator.list_available_ships` e `navigator.travel`.


In [ ]:
# 🔨 SOLUZIONE MISSIONE 1
tool_calls_1 = []  # Traccia le chiamate API

# Trova posizione attuale di R2-D2
droid_location = navigator.get_droid_location('R2-D2')
tool_calls_1.append({'tool': 'get_droid_location', 'droid': 'R2-D2', 'result': droid_location})

# Se non è già su Coruscant, trova nave più economica e viaggia
if droid_location != 'Coruscant':
    ships = navigator.list_available_ships(droid_location)
    # Trova nave più economica
    cheapest_ship = min(ships.values(), key=lambda s: s['rental_cost'])
    travel_result = navigator.travel(droid_location, 'Coruscant', cheapest_ship['name'])
    tool_calls_1.append({'tool': 'travel', 'from': droid_location, 'to': 'Coruscant', 'ship': cheapest_ship['name'], 'result': travel_result})
    agent_response_1 = f"R2-D2 spostato su Coruscant con {cheapest_ship['name']}."
else:
    agent_response_1 = 'R2-D2 era già su Coruscant.'


In [ ]:
# ✅ VALUTAZIONE MISSIONE 1
evaluation_1 = evaluator.evaluate_mission(
    task_id=1,
    agent_response=agent_response_1,
    intermediate_steps=tool_calls_1,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_1, round_number=1)


---

## Missione 2
**Compra un 'Walkman degli Antichi' dal mercato galattico**

Suggerimento: usa `marketplace.find_item` e `marketplace.purchase_item`.


In [ ]:
# 🔨 SOLUZIONE MISSIONE 2
tool_calls_2 = []

# Trova l'oggetto
items = marketplace.find_item('Walkman degli Antichi')
tool_calls_2.append({'tool': 'find_item', 'item': 'Walkman degli Antichi', 'result': items})

if items:
    item_id, item = items[0]
    purchase_result = marketplace.purchase_item(item_id)
    tool_calls_2.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
    agent_response_2 = f"Acquistato Walkman degli Antichi su {item['planet']} per {item['price']} crediti."
else:
    agent_response_2 = 'Oggetto non trovato nel marketplace.'


In [ ]:
# ✅ VALUTAZIONE MISSIONE 2
evaluation_2 = evaluator.evaluate_mission(
    task_id=2,
    agent_response=agent_response_2,
    intermediate_steps=tool_calls_2,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_2, round_number=1)


---

## Missione 3
**Usa l'InfoSfera per scoprire informazioni su 'Cybersystems Inc.'**

Suggerimento: usa `infosphere.get_info`.


In [ ]:
# 🔨 SOLUZIONE MISSIONE 3
tool_calls_3 = []
info = infosphere.get_info('Cybersystems Inc.')
tool_calls_3.append({'tool': 'get_info', 'name': 'Cybersystems Inc.', 'result': info})
agent_response_3 = f"Info su Cybersystems Inc.: {info}"


In [ ]:
# ✅ VALUTAZIONE MISSIONE 3
evaluation_3 = evaluator.evaluate_mission(
    task_id=3,
    agent_response=agent_response_3,
    intermediate_steps=tool_calls_3,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_3, round_number=1)


---

## Missione 4
**Completa una missione combinata: porta R2-D2 su Alderaan E compra almeno 2 oggetti diversi rimanendo nel budget**

Suggerimento: combina le API di navigazione e marketplace.


In [ ]:
# 🔨 SOLUZIONE MISSIONE 4
tool_calls_4 = []
# Esempio: sposta R2-D2 su Alderaan
droid_location = navigator.get_droid_location('R2-D2')
tool_calls_4.append({'tool': 'get_droid_location', 'droid': 'R2-D2', 'result': droid_location})
if droid_location != 'Alderaan':
    ships = navigator.list_available_ships(droid_location)
    cheapest_ship = min(ships.values(), key=lambda s: s['rental_cost'])
    travel_result = navigator.travel(droid_location, 'Alderaan', cheapest_ship['name'])
    tool_calls_4.append({'tool': 'travel', 'from': droid_location, 'to': 'Alderaan', 'ship': cheapest_ship['name'], 'result': travel_result})
    agent_response_4 = f"R2-D2 spostato su Alderaan con {cheapest_ship['name']}."
else:
    agent_response_4 = 'R2-D2 era già su Alderaan.'

# Compra almeno 2 oggetti diversi
items_to_buy = []
for item_name in ['Walkman degli Antichi', 'Crystal Shard']:
    items = marketplace.find_item(item_name)
    if items:
        item_id, item = items[0]
        purchase_result = marketplace.purchase_item(item_id)
        tool_calls_4.append({'tool': 'purchase_item', 'item_id': item_id, 'result': purchase_result})
        items_to_buy.append(item['name'])

agent_response_4 += f" Acquistati: {', '.join(items_to_buy)}."


In [ ]:
# ✅ VALUTAZIONE MISSIONE 4
evaluation_4 = evaluator.evaluate_mission(
    task_id=4,
    agent_response=agent_response_4,
    intermediate_steps=tool_calls_4,
    final_state=galaxy_state
)
display_evaluation_results(evaluation_4, round_number=1)


---

## 🎉 Complimenti!
Hai completato tutte le missioni del Round 1.

- Puoi ottimizzare le tue soluzioni e rieseguire le valutazioni.
- Passa al Round 2 per nuove sfide!
